# quant-econ Solutions: Rational Expectations Equilibrium

Solutions for http://quant-econ.net/jl/rational_expectations.html

The following solutions were put together by Chase Coleman, Spencer Lyon, Thomas Sargent and John Stachurski

In [3]:
using QuantEcon

To map a problem into a [discounted optimal linear control problem](http://quant-econ.net/jl/lqcontrol.html), we need to define

* state vector $x_t$ and control vector $u_t$

* matrices $A, B, Q, R$ that define preferences and the law of motion for the state

For the state and control vectors we choose

$$
    x_t = \begin{bmatrix} y_t \\ Y_t \\ 1 \end{bmatrix},
    \qquad
    u_t = y_{t+1} - y_{t}
$$

For $, B, Q, R$ we set

$$
    A = 
    \begin{bmatrix} 
        1 & 0 & 0 \\
        0 & \kappa_1 & \kappa_0 \\ 
        0 & 0 & 1 
    \end{bmatrix},
    \quad
    B = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix} ,
    \quad
    R = 
    \begin{bmatrix} 
        0 & a_1/2 & -a_0/2 \\ 
        a_1/2 & 0 & 0 \\ 
        -a_0/2 & 0 & 0 
    \end{bmatrix},
    \quad
    Q = \gamma / 2
$$

By multiplying out you can confirm that

* $x_t' R x_t + u_t' Q u_t = - r_t$

* $x_{t+1} = A x_t + B u_t$

We'll use the module  ``lqcontrol.py`` to solve the firm's problem at the stated parameter values

This will return an LQ policy $F$ with the interpretation $u_t = - F x_t$, or

$$
    y_{t+1} - y_t = - F_0 y_t - F_1 Y_t - F_2
$$

Matching parameters with $y_{t+1} = h_0 + h_1 y_t + h_2 Y_t$ leads to

$$
    h_0 = -F_2, \quad h_1 = 1 - F_0, \quad h_2 = -F_1
$$

Here's our solution

In [4]:
# == Model parameters == #
a0      = 100
a1      = 0.05
bet     = 0.95
gamma   = 10.0

# == Beliefs == #
kappa0  = 95.5
kappa1  = 0.95

# == Formulate the LQ problem == #
A = [1 0 0
     0 kappa1 kappa0
     0 0 1]
B = [1.0, 0.0, 0.0]
R = [0 a1/2 -a0/2
     a1/2 0 0
     -a0/2 0 0]
Q = 0.5 * gamma

# == Solve for the optimal policy == #
lq = LQ(Q, R, A, B, nothing, bet)
P, F, d = stationary_values(lq)

hh = h0, h1, h2 = -F[3], 1 - F[1], -F[2]

@printf("F = [%.3f, %.3f, %.3f]\n", F[1], F[2], F[3])
@printf("(h1, h2, h3) = [%.3f, %.3f, %.3f]\n", h0, h1, h2)

F = [-0.000, 0.046, -96.949]
(h1, h2, h3) = [96.949, 1.000, -0.046]


The implication is that

$$
    y_{t+1} = 96.949 + y_t - 0.046 \, Y_t
$$


For the case $n > 1$, recall that $Y_t = n y_t$, which, combined with the previous equation, yields

$$
    Y_{t+1} 
    = n \left( 96.949 + y_t - 0.046 \, Y_t \right)  
    = n 96.949 + (1 - n 0.046) Y_t 
$$

## Exercise 2

To determine whether a $\kappa_0, \kappa_1$ pair forms the
aggregate law of motion component of a rational expectations equilibrium, we
can proceed as follows:

* Determine the corresponding firm law of motion $y_{t+1} = h_0 + h_1 y_t + h_2 Y_t$

* Test whether the associated aggregate law :$Y_{t+1} = n h(Y_t/n, Y_t)$ evaluates to $Y_{t+1} = \kappa_0 + \kappa_1 Y_t$

In the second step we can use $Y_t = n y_t = y_t$, so that $Y_{t+1} = n h(Y_t/n, Y_t)$ becomes

$$
    Y_{t+1} = h(Y_t, Y_t) = h_0 + (h_1 + h_2) Y_t
$$

Hence to test the second step we can test $\kappa_0 = h_0$ and $\kappa_1 = h_1 + h_2$

The following code implements this test


In [3]:
candidates = ([94.0886298678, 0.923409232937],
              [93.2119845412, 0.984323478873],
              [95.0818452486, 0.952459076301])

for (k0, k1) in candidates
    A = [1 0 0
         0 k1 k0
         0 0 1]
    lq = LQ(Q, R, A, B, nothing, bet)
    P, F, d = stationary_values(lq)
    hh = h0, h1, h2 = -F[3], 1 - F[1], -F[2]
    
    if isapprox(k0, h0) && isapprox(k1, h1 + h2)
        @printf("Equilibrium pair= (%.6f, %.6f)\n", k0, k1)
        @printf("(h1, h2, h3) = [%.6f, %.6f, %.6f]\n", h0, h1, h2)
    end
end

Equilibrium pair= (95.081845, 0.952459)
(h1, h2, h3) = [95.081891, 1.000000, -0.047541]


The output tells us that the answer is pair (iii), which implies $(h_0, h_1, h_2) = (95.0819, 1.0000, -.0475)$

(Notice we use `np.allclose` to test equality of floating point numbers, since exact equality is too strict)

Regarding the iterative algorithm, one could loop from a given
$(\kappa_0, \kappa_1)$ pair to the associated firm law and then to a new $(\kappa_0, \kappa_1)$ pair

This amounts to implementing the operator $\Phi$ described in the lecture

(There is in general no guarantee that this iterative process will converge to
a rational expectations equilibrium)


## Exercise 3

We are asked to write the planner problem as an LQ problem 

For the state and control vectors we choose

$$
    x_t = \begin{bmatrix} Y_t \\ 1 \end{bmatrix},
    \quad
    u_t = Y_{t+1} - Y_{t}
$$

For the LQ matrices we set

$$
    A = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix},
    \quad
    B = \begin{bmatrix} 1 \\ 0 \end{bmatrix},
    \quad
    R = \begin{bmatrix} a_1/2 & -a_0/2 \\ -a_0/2 & 0 \end{bmatrix},
    \quad
    Q = \gamma / 2
$$

By multiplying out you can confirm that

* $x_t' R x_t + u_t' Q u_t = - s(Y_t, Y_{t+1})$

* $x_{t+1} = A x_t + B u_t$

By obtaining the optimal policy and using $u_t = - F x_t$ or

$$
    Y_{t+1} - Y_t = -F_0 Y_t - F_1 
$$

we can obtain the implied aggregate law of motion via $\kappa_0 = -F_1$
and $\kappa_1 = 1-F_0$

The Python code to solve this problem is below:



In [4]:
# == Formulate the planner's LQ problem == #
A = eye(2)
B = [1.0, 0.0]
R = [a1 / 2.0 -a0 / 2.0
     -a0 / 2.0 0.0]
Q = gamma / 2.0

# == Solve for the optimal policy == #
lq = LQ(Q, R, A, B, nothing, bet)
P, F, d = stationary_values(lq)

# == Print the results == #
kappa0, kappa1 = -F[2], 1 - F[1]
println("kappa0=$kappa0\tkappa1=$kappa1")

kappa0=95.08187459215024	kappa1=0.9524590627039249


The output yields the same $(\kappa_0, \kappa_1)$ pair obtained as an equilibrium from the previous exercise


## Exercise 4

The monopolist's LQ problem is almost identical to the planner's problem from
the previous exercise, except that

$$
    R = \begin{bmatrix} 
        a_1 & -a_0/2 \\ 
        -a_0/2 & 0 
    \end{bmatrix} 
$$

The problem can be solved as follows


In [5]:
# == Formulate the monopolist's LQ problem == #
A = eye(2)
B = [1.0, 0.0]
R = [a1 -a0 / 2.0
     -a0 / 2.0 0.0]
Q = gamma / 2.0

# == Solve for the optimal policy == #
lq = LQ(Q, R, A, B, nothing, bet)
P, F, d = stationary_values(lq)

# == Print the results == #
m0, m1 = -F[2], 1 - F[1]
println("m0=$m0\tm1=$m1")

m0=73.47294403502895	m1=0.9265270559649711


We see that the law of motion for the monopolist is approximately $Y_{t+1} = 73.4729 + 0.9265 Y_t$

In the rational expectations case the law of motion was approximately
$Y_{t+1} = 95.0818 + 0.9525 Y_t$

One way to compare these two laws of motion is by their fixed points, which give long run equilibrium output in each case

For laws of the form $Y_{t+1} = c_0 + c_1 Y_t$, the fixed point is $c_0 / (1 - c_1)$

If you crunch the numbers, you will see that the monopolist adopts a lower long run
quantity than obtained by the competitive market, implying a higher market price

This is analogous to the elementary static-case results
